# Partie 3.2 - Analyse des correlations

In [ ]:
# --- Imports et chargement des donnees ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configuration graphique
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['figure.dpi'] = 100
sns.set_theme(style='whitegrid', palette='muted')

# Chargement du fichier enrichi
df = pd.read_parquet('../output/consommations_enrichies.parquet')

print(f"Dimensions du jeu de donnees : {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f"\nColonnes disponibles :\n{list(df.columns)}")
print(f"\nTypes d'energie : {df['type_energie'].unique().tolist()}")
df.head()

## 1. Preparation des donnees pour la correlation

In [ ]:
# --- Selection des colonnes numeriques pertinentes ---
colonnes_numeriques = [
    'consommation', 'temperature_c', 'humidite_pct',
    'rayonnement_solaire_wm2', 'vitesse_vent_kmh', 'precipitation_mm',
    'surface_m2', 'nb_occupants_moyen', 'annee_construction',
    'cout', 'consommation_par_occupant', 'consommation_par_m2'
]

# Verification de la presence des colonnes dans le DataFrame
colonnes_presentes = [col for col in colonnes_numeriques if col in df.columns]
colonnes_manquantes = [col for col in colonnes_numeriques if col not in df.columns]
if colonnes_manquantes:
    print(f"Colonnes manquantes (ignorees) : {colonnes_manquantes}")
print(f"Colonnes retenues pour l'analyse : {colonnes_presentes}")

# --- Creation d'un pivot pour separer les consommations par type d'energie ---
# Identifiant unique pour chaque ligne : on utilise l'index original
# On cree des colonnes distinctes : conso_electricite, conso_gaz, conso_eau
colonnes_id = [col for col in df.columns if col not in ['consommation', 'type_energie', 'cout',
                                                          'consommation_par_occupant', 'consommation_par_m2']]

# Pivot : une ligne par observation avec les consommations eclatees par type
df_pivot = df.pivot_table(
    index=[col for col in ['date', 'batiment_id'] if col in df.columns],
    columns='type_energie',
    values='consommation',
    aggfunc='mean'
).reset_index()

# Renommage des colonnes pivotees
df_pivot.columns = [f'conso_{col}' if col in ['electricite', 'gaz', 'eau'] else col
                     for col in df_pivot.columns]

# Fusion avec les variables meteo et batiment (on prend les valeurs uniques par date/batiment)
colonnes_contexte = ['temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2',
                      'vitesse_vent_kmh', 'precipitation_mm', 'surface_m2',
                      'nb_occupants_moyen', 'annee_construction']
colonnes_contexte = [col for col in colonnes_contexte if col in df.columns]

colonnes_merge = [col for col in ['date', 'batiment_id'] if col in df.columns]
df_contexte = df.groupby(colonnes_merge)[colonnes_contexte].first().reset_index()

df_corr = df_pivot.merge(df_contexte, on=colonnes_merge, how='left')

# Echantillonnage si le jeu est trop volumineux (> 50 000 lignes)
SEUIL_ECHANTILLONNAGE = 50_000
if len(df_corr) > SEUIL_ECHANTILLONNAGE:
    print(f"Echantillonnage de {SEUIL_ECHANTILLONNAGE} lignes sur {len(df_corr)} pour le calcul de correlation.")
    df_corr = df_corr.sample(n=SEUIL_ECHANTILLONNAGE, random_state=42)
else:
    print(f"Nombre de lignes pour l'analyse de correlation : {len(df_corr)}")

print(f"\nColonnes du DataFrame pivot :\n{list(df_corr.columns)}")
df_corr.head()

## 2. Matrice de correlation globale

In [ ]:
# --- Calcul de la matrice de correlation de Pearson ---
# Selection des colonnes numeriques pour la matrice
colonnes_corr = [
    'conso_electricite', 'conso_gaz', 'conso_eau',
    'temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh',
    'precipitation_mm',
    'surface_m2', 'nb_occupants_moyen', 'annee_construction'
]
colonnes_corr = [col for col in colonnes_corr if col in df_corr.columns]

matrice_corr = df_corr[colonnes_corr].corr(method='pearson')

# Affichage formate
print("=" * 80)
print("MATRICE DE CORRELATION DE PEARSON")
print("=" * 80)
display(matrice_corr.style.background_gradient(cmap='RdBu_r', vmin=-1, vmax=1)
        .format('{:.3f}'))

# Heatmap
fig, ax = plt.subplots(figsize=(14, 10))
masque = np.triu(np.ones_like(matrice_corr, dtype=bool))
sns.heatmap(
    matrice_corr,
    mask=masque,
    annot=True,
    fmt='.2f',
    cmap='RdBu_r',
    vmin=-1, vmax=1,
    center=0,
    square=True,
    linewidths=0.5,
    ax=ax
)
ax.set_title('Matrice de correlation - Variables numeriques', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. Correlations significatives

In [ ]:
# --- Filtrage des correlations significatives (|r| > 0.5) ---

# Extraction des paires uniques de la matrice (triangle inferieur)
correlations_significatives = []

for i in range(len(matrice_corr.columns)):
    for j in range(i + 1, len(matrice_corr.columns)):
        r = matrice_corr.iloc[i, j]
        if abs(r) > 0.5:
            correlations_significatives.append({
                'Variable_1': matrice_corr.columns[i],
                'Variable_2': matrice_corr.columns[j],
                'Correlation': round(r, 4),
                'Correlation_abs': round(abs(r), 4),
                'Direction': 'Positive' if r > 0 else 'Negative'
            })

df_sign = pd.DataFrame(correlations_significatives)

if len(df_sign) > 0:
    df_sign = df_sign.sort_values('Correlation_abs', ascending=False).reset_index(drop=True)

    print("=" * 80)
    print("CORRELATIONS SIGNIFICATIVES (|r| > 0.5)")
    print("=" * 80)
    display(df_sign)

    # --- Interpretation de chaque correlation significative ---
    print("\n" + "=" * 80)
    print("INTERPRETATION DES CORRELATIONS SIGNIFICATIVES")
    print("=" * 80)

    interpretations = {
        ('conso_gaz', 'temperature_c'): "Correlation negative attendue : le chauffage au gaz augmente quand la temperature baisse.",
        ('conso_electricite', 'rayonnement_solaire_wm2'): "L'ensoleillement peut etre lie a la climatisation en ete ou a la reduction de l'eclairage.",
        ('conso_electricite', 'temperature_c'): "La consommation electrique peut augmenter en ete (climatisation) ou en hiver (chauffage electrique).",
        ('conso_gaz', 'humidite_pct'): "L'humidite elevee en hiver est associee a un besoin de chauffage accru.",
        ('surface_m2', 'nb_occupants_moyen'): "Les grands batiments accueillent naturellement plus d'occupants.",
        ('conso_electricite', 'surface_m2'): "Plus la surface est grande, plus la consommation electrique est elevee.",
        ('conso_gaz', 'surface_m2'): "Plus la surface est grande, plus le volume a chauffer est important.",
        ('conso_electricite', 'nb_occupants_moyen'): "Plus d'occupants implique plus d'equipements electriques en fonctionnement.",
        ('conso_eau', 'nb_occupants_moyen'): "La consommation d'eau est directement liee au nombre d'utilisateurs.",
        ('temperature_c', 'rayonnement_solaire_wm2'): "Le rayonnement solaire et la temperature sont naturellement correles (meteo).",
    }

    for _, row in df_sign.iterrows():
        v1, v2 = row['Variable_1'], row['Variable_2']
        cle = (v1, v2) if (v1, v2) in interpretations else (v2, v1)
        interpretation = interpretations.get(cle, "Correlation a investiguer selon le contexte metier.")
        direction = "positive" if row['Direction'] == 'Positive' else "negative"
        print(f"\n  {v1} <-> {v2} : r = {row['Correlation']:.3f} ({direction})")
        print(f"    -> {interpretation}")
else:
    print("Aucune correlation significative (|r| > 0.5) detectee.")

## 4. Impact de la temperature sur la consommation de chauffage (gaz)

In [ ]:
# --- Analyse de la relation temperature / consommation de gaz ---

# Filtrage des donnees gaz
df_gaz = df[df['type_energie'] == 'gaz'].copy()
print(f"Nombre d'observations gaz : {len(df_gaz)}")

# Creation de tranches de temperature
bins_temp = [-10, 0, 5, 10, 15, 20, 25, 30, 40]
labels_temp = ['< 0', '0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '> 30']
df_gaz['tranche_temperature'] = pd.cut(df_gaz['temperature_c'], bins=bins_temp, labels=labels_temp)

# Consommation moyenne par tranche de temperature
conso_par_temp = df_gaz.groupby('tranche_temperature', observed=True).agg(
    consommation_moyenne=('consommation', 'mean'),
    consommation_mediane=('consommation', 'median'),
    nombre_observations=('consommation', 'count')
).reset_index()

print("\n" + "=" * 60)
print("CONSOMMATION GAZ MOYENNE PAR TRANCHE DE TEMPERATURE")
print("=" * 60)
display(conso_par_temp)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1 : barres - consommation moyenne par tranche
axes[0].bar(
    conso_par_temp['tranche_temperature'].astype(str),
    conso_par_temp['consommation_moyenne'],
    color=sns.color_palette('coolwarm', len(conso_par_temp)),
    edgecolor='black', linewidth=0.5
)
axes[0].set_xlabel('Tranche de temperature (°C)', fontsize=12)
axes[0].set_ylabel('Consommation gaz moyenne', fontsize=12)
axes[0].set_title('Consommation gaz par tranche de temperature', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

# Graphique 2 : nuage de points avec regression
echantillon_gaz = df_gaz.sample(n=min(5000, len(df_gaz)), random_state=42)
axes[1].scatter(
    echantillon_gaz['temperature_c'],
    echantillon_gaz['consommation'],
    alpha=0.2, s=10, color='steelblue'
)
# Droite de regression
z = np.polyfit(df_gaz['temperature_c'].dropna(), df_gaz['consommation'].dropna(), 1)
p = np.poly1d(z)
x_line = np.linspace(df_gaz['temperature_c'].min(), df_gaz['temperature_c'].max(), 100)
axes[1].plot(x_line, p(x_line), color='red', linewidth=2, label='Regression lineaire')
axes[1].set_xlabel('Temperature (°C)', fontsize=12)
axes[1].set_ylabel('Consommation gaz', fontsize=12)
axes[1].set_title('Temperature vs Consommation gaz', fontsize=14, fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

# --- Calcul du R-squared ---
from scipy import stats

mask_valid = df_gaz[['temperature_c', 'consommation']].dropna()
slope, intercept, r_value, p_value, std_err = stats.linregress(
    mask_valid['temperature_c'], mask_valid['consommation']
)
r_squared = r_value ** 2

print(f"\n--- Regression lineaire : Temperature -> Consommation gaz ---")
print(f"  Pente          : {slope:.4f}")
print(f"  Ordonnee       : {intercept:.4f}")
print(f"  R              : {r_value:.4f}")
print(f"  R-squared      : {r_squared:.4f}")
print(f"  p-value        : {p_value:.2e}")

if slope < 0:
    print("\n  => Relation INVERSE confirmee : quand la temperature baisse, la consommation gaz augmente.")
    print(f"     En moyenne, chaque degre de moins entraine +{abs(slope):.2f} unites de consommation gaz.")
else:
    print("\n  => Relation directe : la consommation gaz augmente avec la temperature (inattendu).")

## 5. Effet du rayonnement solaire sur la consommation electrique

In [ ]:
# --- Analyse du rayonnement solaire vs consommation electrique ---

df_elec = df[df['type_energie'] == 'electricite'].copy()
print(f"Nombre d'observations electricite : {len(df_elec)}")

# --- Analyse saisonniere ---
# Attribution d'une saison basee sur le mois
if 'date' in df_elec.columns:
    df_elec['date'] = pd.to_datetime(df_elec['date'])
    df_elec['mois'] = df_elec['date'].dt.month
    df_elec['saison'] = df_elec['mois'].map({
        12: 'Hiver', 1: 'Hiver', 2: 'Hiver',
        3: 'Printemps', 4: 'Printemps', 5: 'Printemps',
        6: 'Ete', 7: 'Ete', 8: 'Ete',
        9: 'Automne', 10: 'Automne', 11: 'Automne'
    })

# Creation de tranches de rayonnement solaire
bins_solaire = [0, 100, 200, 400, 600, 800, 1200]
labels_solaire = ['0-100', '100-200', '200-400', '400-600', '600-800', '800+']
df_elec['tranche_solaire'] = pd.cut(
    df_elec['rayonnement_solaire_wm2'], bins=bins_solaire, labels=labels_solaire
)

# Consommation electrique par tranche de rayonnement
conso_par_solaire = df_elec.groupby('tranche_solaire', observed=True).agg(
    consommation_moyenne=('consommation', 'mean'),
    consommation_mediane=('consommation', 'median'),
    nombre_observations=('consommation', 'count')
).reset_index()

print("\n" + "=" * 60)
print("CONSOMMATION ELECTRIQUE PAR TRANCHE DE RAYONNEMENT SOLAIRE")
print("=" * 60)
display(conso_par_solaire)

# --- Analyse par saison ---
if 'saison' in df_elec.columns:
    conso_saison_solaire = df_elec.groupby(['saison', 'tranche_solaire'], observed=True).agg(
        consommation_moyenne=('consommation', 'mean'),
        nombre_observations=('consommation', 'count')
    ).reset_index()

    print("\nConsommation electrique par saison et rayonnement solaire :")
    display(conso_saison_solaire)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Barres : consommation par tranche de rayonnement
axes[0].bar(
    conso_par_solaire['tranche_solaire'].astype(str),
    conso_par_solaire['consommation_moyenne'],
    color=sns.color_palette('YlOrRd', len(conso_par_solaire)),
    edgecolor='black', linewidth=0.5
)
axes[0].set_xlabel('Rayonnement solaire (W/m²)', fontsize=12)
axes[0].set_ylabel('Consommation electrique moyenne', fontsize=12)
axes[0].set_title('Consommation electrique par rayonnement solaire', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

# Nuage de points par saison
if 'saison' in df_elec.columns:
    couleurs_saison = {'Hiver': 'blue', 'Printemps': 'green', 'Ete': 'orange', 'Automne': 'brown'}
    echantillon_elec = df_elec.sample(n=min(5000, len(df_elec)), random_state=42)
    for saison, couleur in couleurs_saison.items():
        mask = echantillon_elec['saison'] == saison
        axes[1].scatter(
            echantillon_elec.loc[mask, 'rayonnement_solaire_wm2'],
            echantillon_elec.loc[mask, 'consommation'],
            alpha=0.3, s=10, color=couleur, label=saison
        )
    axes[1].legend(title='Saison')
else:
    echantillon_elec = df_elec.sample(n=min(5000, len(df_elec)), random_state=42)
    axes[1].scatter(
        echantillon_elec['rayonnement_solaire_wm2'],
        echantillon_elec['consommation'],
        alpha=0.2, s=10, color='darkorange'
    )

axes[1].set_xlabel('Rayonnement solaire (W/m²)', fontsize=12)
axes[1].set_ylabel('Consommation electrique', fontsize=12)
axes[1].set_title('Rayonnement solaire vs Consommation electrique', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Correlation globale rayonnement / electricite
r_solaire = df_elec[['rayonnement_solaire_wm2', 'consommation']].dropna()
r_val = r_solaire['rayonnement_solaire_wm2'].corr(r_solaire['consommation'])
print(f"\nCorrelation rayonnement solaire <-> consommation electrique : r = {r_val:.4f}")

## 6. Correlations par type de batiment

In [ ]:
# --- Matrice de correlation par type de batiment ---

# Detection de la colonne type de batiment
col_type_bat = None
for candidate in ['type_batiment', 'type_bat', 'categorie_batiment']:
    if candidate in df.columns:
        col_type_bat = candidate
        break

if col_type_bat is None:
    print("Aucune colonne de type de batiment trouvee. Analyse ignoree.")
else:
    types_batiment = df[col_type_bat].unique()
    print(f"Types de batiment detectes : {types_batiment.tolist()}")

    # Variables cles pour l'analyse de sensibilite meteo
    vars_meteo = ['temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2',
                  'vitesse_vent_kmh', 'precipitation_mm']
    vars_meteo = [v for v in vars_meteo if v in df.columns]

    # Calcul de la correlation consommation <-> variables meteo pour chaque type
    resultats_sensibilite = []

    nb_types = len(types_batiment)
    fig, axes = plt.subplots(1, min(nb_types, 4), figsize=(6 * min(nb_types, 4), 5))
    if nb_types == 1:
        axes = [axes]

    for idx, type_bat in enumerate(types_batiment[:4]):
        df_type = df[df[col_type_bat] == type_bat]
        colonnes_analyse = ['consommation'] + vars_meteo
        colonnes_analyse = [c for c in colonnes_analyse if c in df_type.columns]

        corr_type = df_type[colonnes_analyse].corr()

        # Heatmap
        sns.heatmap(
            corr_type, annot=True, fmt='.2f', cmap='RdBu_r',
            vmin=-1, vmax=1, center=0, ax=axes[idx],
            square=True, linewidths=0.5
        )
        axes[idx].set_title(f'{type_bat}', fontsize=12, fontweight='bold')

        # Sensibilite meteo : correlation absolue moyenne avec la consommation
        if 'consommation' in corr_type.index:
            sensibilite = corr_type.loc['consommation', vars_meteo].abs().mean()
            resultats_sensibilite.append({
                'type_batiment': type_bat,
                'sensibilite_meteo_moyenne': round(sensibilite, 4),
                'correlation_temperature': round(
                    corr_type.loc['consommation', 'temperature_c'] if 'temperature_c' in corr_type.columns else 0, 4
                )
            })

    plt.suptitle('Correlations par type de batiment', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    # Classement des types de batiment par sensibilite meteo
    if resultats_sensibilite:
        df_sensibilite = pd.DataFrame(resultats_sensibilite).sort_values(
            'sensibilite_meteo_moyenne', ascending=False
        ).reset_index(drop=True)

        print("\n" + "=" * 60)
        print("SENSIBILITE METEO PAR TYPE DE BATIMENT")
        print("(correlation absolue moyenne entre consommation et variables meteo)")
        print("=" * 60)
        display(df_sensibilite)

        type_plus_sensible = df_sensibilite.iloc[0]['type_batiment']
        print(f"\n=> Le type de batiment le plus sensible a la meteo : {type_plus_sensible}")

## 7. Correlation temporelle

In [ ]:
# --- Evolution des correlations au fil des saisons et des mois ---

df_temp = df.copy()
if 'date' in df_temp.columns:
    df_temp['date'] = pd.to_datetime(df_temp['date'])
    df_temp['mois'] = df_temp['date'].dt.month
    df_temp['saison'] = df_temp['mois'].map({
        12: 'Hiver', 1: 'Hiver', 2: 'Hiver',
        3: 'Printemps', 4: 'Printemps', 5: 'Printemps',
        6: 'Ete', 7: 'Ete', 8: 'Ete',
        9: 'Automne', 10: 'Automne', 11: 'Automne'
    })

# --- Correlation mensuelle : temperature vs consommation gaz ---
df_gaz_temp = df_temp[df_temp['type_energie'] == 'gaz'].copy()

if 'mois' in df_gaz_temp.columns:
    correlations_mensuelles = []
    for mois in range(1, 13):
        df_mois = df_gaz_temp[df_gaz_temp['mois'] == mois]
        if len(df_mois) > 10:
            r = df_mois['temperature_c'].corr(df_mois['consommation'])
            correlations_mensuelles.append({
                'mois': mois,
                'nom_mois': ['Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Jun',
                             'Jul', 'Aou', 'Sep', 'Oct', 'Nov', 'Dec'][mois - 1],
                'correlation_temp_gaz': round(r, 4),
                'nb_observations': len(df_mois)
            })

    df_corr_mensuelle = pd.DataFrame(correlations_mensuelles)

    print("=" * 60)
    print("CORRELATION MENSUELLE : TEMPERATURE vs CONSOMMATION GAZ")
    print("=" * 60)
    display(df_corr_mensuelle)

    # Visualisation
    fig, ax = plt.subplots(figsize=(12, 5))
    couleurs = ['blue' if v < 0 else 'red' for v in df_corr_mensuelle['correlation_temp_gaz']]
    ax.bar(
        df_corr_mensuelle['nom_mois'],
        df_corr_mensuelle['correlation_temp_gaz'],
        color=couleurs, edgecolor='black', linewidth=0.5
    )
    ax.axhline(y=0, color='black', linewidth=0.8)
    ax.axhline(y=-0.5, color='gray', linewidth=0.5, linestyle='--', label='Seuil |0.5|')
    ax.axhline(y=0.5, color='gray', linewidth=0.5, linestyle='--')
    ax.set_xlabel('Mois', fontsize=12)
    ax.set_ylabel('Coefficient de correlation (r)', fontsize=12)
    ax.set_title('Evolution mensuelle de la correlation Temperature / Gaz', fontsize=14, fontweight='bold')
    ax.set_ylim(-1, 1)
    ax.legend()
    plt.tight_layout()
    plt.show()

    # --- Correlations par saison ---
    print("\n" + "=" * 60)
    print("CORRELATIONS PAR SAISON")
    print("=" * 60)

    for saison in ['Hiver', 'Printemps', 'Ete', 'Automne']:
        df_saison = df_temp[df_temp['saison'] == saison]
        if len(df_saison) > 10:
            vars_analyse = ['consommation', 'temperature_c', 'rayonnement_solaire_wm2', 'humidite_pct']
            vars_analyse = [v for v in vars_analyse if v in df_saison.columns]
            corr_saison = df_saison[vars_analyse].corr().loc['consommation'].drop('consommation')
            print(f"\n  {saison} ({len(df_saison)} obs.) :")
            for var, val in corr_saison.items():
                print(f"    {var:35s} : r = {val:+.4f}")
else:
    print("Colonne 'date' absente : analyse temporelle impossible.")

## 8. Export et synthese

In [ ]:
# --- Export de la matrice de correlation ---
chemin_export = '../output/matrice_correlation.csv'
matrice_corr.to_csv(chemin_export)
print(f"Matrice de correlation exportee vers : {chemin_export}")

# --- Synthese en Markdown ---
from IPython.display import Markdown, display as md_display

synthese = """
### Synthese des resultats de l'analyse de correlations

| Analyse | Resultat principal |
|---------|--------------------|
| **Matrice globale** | Correlations calculees entre {n_vars} variables numeriques |
| **Correlations significatives** | {n_sign} paires avec \|r\| > 0.5 identifiees |
| **Temperature vs Gaz** | R² = {r2:.4f} — relation inverse confirmee (froid = plus de chauffage) |
| **Solaire vs Electricite** | r = {r_sol:.4f} — impact du rayonnement sur la consommation electrique |
| **Sensibilite batiment** | Sensibilite meteo variable selon le type de batiment |
| **Temporalite** | Correlations plus fortes en hiver qu'en ete pour le gaz |

**Fichier exporte** : `matrice_correlation.csv`
""".format(
    n_vars=len(colonnes_corr),
    n_sign=len(df_sign) if 'df_sign' in dir() and len(df_sign) > 0 else 0,
    r2=r_squared if 'r_squared' in dir() else 0.0,
    r_sol=r_val if 'r_val' in dir() else 0.0
)

md_display(Markdown(synthese))
print("\nAnalyse des correlations terminee avec succes.")

## Synthese des insights

**Principaux enseignements de l'analyse de correlations :**

- **Temperature et gaz** : La temperature exterieure est le facteur le plus influent sur la consommation de gaz (chauffage). La relation est inversement proportionnelle — plus il fait froid, plus on consomme de gaz. Le R-squared confirme la significativite statistique de cette relation.

- **Rayonnement solaire et electricite** : L'ensoleillement impacte la consommation electrique, avec des effets saisonniers distincts (climatisation en ete, eclairage reduit en journee).

- **Surface et consommation** : Les batiments de grande surface consomment logiquement davantage toutes energies confondues. La surface est un facteur structurel majeur.

- **Nombre d'occupants** : Le nombre d'occupants est fortement correle a la consommation d'eau et d'electricite, confirmant l'impact direct de l'usage humain.

- **Sensibilite par type de batiment** : Certains types de batiments sont beaucoup plus sensibles aux conditions meteorologiques que d'autres, ce qui oriente les strategies de renovation energetique.

- **Saisonnalite des correlations** : Les correlations entre variables meteo et consommation varient significativement selon la saison, ce qui souligne l'importance d'une analyse temporelle fine.

- **Variables meteorologiques correlees** : Attention aux multi-colinearites entre variables meteo (temperature/rayonnement solaire) qui peuvent fausser les modeles predictifs.

Ces resultats orientent la suite de l'analyse vers la modelisation predictive en identifiant les variables les plus pertinentes.